In [19]:
import qiime2
import os
os.getcwd()
#change working dir to above
os.chdir('/home/rtsantos3/bioinfo_pipelines/picrust2-2.4.2/data/Kiara-Qiime2/')

In [15]:
!qiime tools import   \
--type 'SampleData[PairedEndSequencesWithQuality]'   \
--input-path 'read-files/sample-manifest.tsv'   \
--output-path paired-end-demux.qza   \
--input-format PairedEndFastqManifestPhred33V2 \



There was a problem importing read-files/sample-manifest.tsv:

  Importing 'SingleLanePerSamplePairedEndFastqDirFmt' requires a directory, not read-files/sample-manifest.tsv



In [ ]:
# #Denoising Step

# !qiime dada2 denoise-single \
#     --i-demultiplexed-seqs paired-end-demux.qza \
#     --p-trunc-len 140 \
#     --p-n-threads 2 \
#     --output-dir dada2 \ 
#     --verbose

In [71]:
!qiime demux summarize \
    --i-data paired-end-demux.qza \
    --o-visualization demux.qzv

Saved Visualization to: demux.qzv


In [74]:
#Denoising Step (paired )

!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs paired-end-demux.qza \
    --p-trunc-len-f 300 \
    --p-trunc-len-r 220 \
    --p-n-threads 6 \
    --p-pooling-method "independent" \
    --output-dir dada2 \
    --verbose

#Analyzing paired sequences does not produce any proper results. Bakit kaya?
#Oks na I just needed to keep lengths at around 300


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmpu79org_v/forward /tmp/tmpu79org_v/reverse /tmp/tmpu79org_v/output.tsv.biom /tmp/tmpu79org_v/track.tsv /tmp/tmpu79org_v/filt_f /tmp/tmpu79org_v/filt_r 300 220 0 0 2.0 2.0 2 12 independent consensus 1.0 6 1000000

R version 4.0.5 (2021-03-31) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering ....
2) Learning Error Rates
67743600 total bases in 225812 reads from 4 samples will be used for learning the error rates.
49678640 total bases in 225812 reads from 4 samples will be used for learning the error rates.
3) Denoise samples ....
....
4) Remove chimeras (method = consensus)
6) Write output
Saved FeatureTable[Frequency] to: dada2/table.qza
Saved FeatureData[Sequence] to: dada2/re

In [75]:

!qiime feature-classifier classify-consensus-vsearch \
        --i-query "dada2/representative_sequences.qza" \
        --i-reference-reads "../training-sets/silva-138-99-seqs.qza" \
        --i-reference-taxonomy "../training-sets/silva-138-99-tax.qza" \
        --output-dir "taxonomy_vsearch" \
        --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-ft7yykkk/bd99ad4e-70d1-4d13-beab-18a81dd5ea00/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2-archive-_1k_nwcu/a7432d0f-b5f7-409f-9daf-cd33db5de53f/data/dna-sequences.fasta --threads 1 --output_no_hits --blast6out /tmp/tmp1fyjtj87

vsearch v2.7.0_linux_x86_64, 15.3GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-_1k_nwcu/a7432d0f-b5f7-409f-9daf-cd33db5de53f/data/dna-sequences.fasta 100%                          
636194718 nt in 436680 seqs, min 900, max 3983, avg 1457
Masking 100%                                                                                                                                         

In [77]:
!qiime tools export \
    --input-path "taxonomy_vsearch/classification.qza" \
    --output-path "taxonomy_vsearch/"


Exported taxonomy_vsearch/classification.qza as TSVTaxonomyDirectoryFormat to directory taxonomy_vsearch/


In [78]:
!qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising-stats.qzv

Saved Visualization to: dada2/denoising-stats.qzv


In [79]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences "dada2/representative_sequences.qza" \
    --output-dir mafft-tree

Saved FeatureData[AlignedSequence] to: mafft-tree/alignment.qza
Saved FeatureData[AlignedSequence] to: mafft-tree/masked_alignment.qza
Saved Phylogeny[Unrooted] to: mafft-tree/tree.qza
Saved Phylogeny[Rooted] to: mafft-tree/rooted_tree.qza


In [80]:
#This code block is to filter out any Eukaryotic sequences from the feature table (Paired)
!qiime taxa filter-table \
  --i-table 'dada2/table.qza' \
  --i-taxonomy 'taxonomy_vsearch/classification.qza' \
  --p-mode exact \
  --p-exclude "Unassigned" \
  --o-filtered-table 'dada2/filtered_table.qza'



Saved FeatureTable[Frequency] to: dada2/filtered_table.qza


In [91]:

os.mkdir('plots')
#For validating paired by generating taxa barplot


!qiime taxa barplot  \
    --i-table 'dada2/filtered_table.qza' \
    --i-taxonomy 'taxonomy_vsearch/classification.qza' \
    --m-metadata-file './read-files/metadata.tsv' \
    --o-visualization 'plots/visualization' \
    --verbose 



Saved Visualization to: plots/visualization.qzv


These code-blocks below is to compute both the Alpha and Beta Rarefaction curves as well as other diversity stats using the filtered  table

In [94]:
#Alpha Diversity stat


!qiime diversity core-metrics-phylogenetic \
    --i-table './dada2/filtered_table.qza' \
    --i-phylogeny './mafft-tree/rooted_tree.qza' \
    --p-sampling-depth 5000 \
    --m-metadata-file './read-files/metadata.tsv' \
    --output-dir diversity

Saved FeatureTable[Frequency] to: diversity/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: diversity/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: diversity/evenness_vector.qza
Saved DistanceMatrix to: diversity/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: diversity/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: diversity/jaccard_distance_matrix.qza
Saved DistanceMatrix to: diversity/bray_curtis_distance_matrix.qza
Saved PCoAResults to: diversity/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity/jaccard_pcoa_results.qza
Saved PCoAResults to: diversity/bray_curtis_pcoa_results.qza
Saved Visualization to: diversity/unweighted_unifrac_emperor.qzv
Saved Visualization to: diversity/weighted_unifrac_emperor.qzv
Saved

In [96]:
!qiime diversity alpha-rarefaction \
    --i-table 'dada2/filtered_table.qza' \
    --i-phylogeny './mafft-tree/rooted_tree.qza' \
    --p-max-depth 4970 \
    --p-metrics 'shannon' 'fisher_alpha' 'chao1' 'goods_coverage' 'simpson_e' 'observed_features' \
    --m-metadata-file './read-files/metadata.tsv' \
    --output-dir './diversity/alpha-rarefaction/' \
    --verbose

Saved Visualization to: ./diversity/alpha-rarefaction/visualization.qzv


In [97]:
!qiime diversity beta-rarefaction \
    --i-table './dada2/filtered_table.qza' \
    --i-phylogeny './mafft-tree/rooted_tree.qza' \
    --p-metric 'weighted_unifrac' \
    --p-clustering-method 'upgma' \
    --m-metadata-file './read-files/metadata.tsv' \
    --p-sampling-depth 4970 \
    --p-correlation-method 'spearman' \
    --output-dir './diversity/beta-diversity' \
    --verbose

Saved Visualization to: ./diversity/beta-diversity/visualization.qzv


In [102]:
!qiime empress tree-plot \
    --i-tree mafft-tree/rooted_tree.qza \
    --o-visualization mafft-tree/empress.qzv

Saved Visualization to: mafft-tree/empress.qzv


In [99]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file 'read-files/metadata.tsv' \
    --o-visualization diversity/alpha_groups.qzv

Saved Visualization to: diversity/alpha_groups.qzv


In [103]:
#Permanova via Adonis addon

!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file read-files/metadata.tsv \
    --p-formula "treatment" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

Saved Visualization to: diversity/permanova.qzv


In [105]:
#Export feature table as well as representative sequences for Picrust2
os.mkdir('picrust2-strat')

!qiime tools export \
    --input-path "dada2/representative_sequences.qza" \
    --output-path "picrust2-strat/dna-sequences.fasta"

!qiime tools export \
    --input-path  "dada2/filtered_table.qza" \
    --output-path "picrust2-strat/feature-table.biom"

Exported dada2/representative_sequences.qza as DNASequencesDirectoryFormat to directory picrust2-strat/dna-sequences.fasta
Exported dada2/filtered_table.qza as BIOMV210DirFmt to directory picrust2-strat/feature-table.biom
